In [1]:
!pip install dgl --quiet

In [2]:
import pandas as pd
import numpy as np

import time
import os
os.environ["DGLBACKEND"] = "pytorch"

import tqdm
import math
import socket
import struct
import random

import dgl
import dgl.nn as dglnn
import dgl.function as fn
from dgl import from_networkx
import networkx as nx

import torch as th
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler, Normalizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report
import category_encoders as ce

import seaborn as sns
import matplotlib.pyplot as plt

residual = True # adding edge features

num_epochs = 10
batch_size = 128
learning_rate = 0.001
LAMBD_1 = 0.0001
LAMBD_2 = 0.001

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cic-ton-iot-parquet/cic_ton_iot.parquet
/kaggle/input/cic-ids-2017-parquet/cic_ids_2017.parquet


In [3]:
df = pd.read_parquet("/kaggle/input/cic-ids-2017-parquet/cic_ids_2017.parquet")
# df = pd.read_parquet("/kaggle/input/cic-ton-iot-parquet/cic_ton_iot.parquet")
# df = pd.read_parquet("/kaggle/input/nf-cse-cic-ids2018")
#df = pd.read_parquet("/kaggle/input/nf-uq-nids")
#df = pd.read_csv("/kaggle/input/ddos-sdn/dataset_sdn.csv")


name = "cic_ids_2017"
# name = "cic_ton_iot"
# name = "nf-cse-cic-2018"

In [4]:
df = df.sample(frac=0.5, random_state=42)  # 10% sample, random_state for reproducibility

In [5]:
# _, stratified_sample = train_test_split(df, test_size=0.05, stratify=df['Label'])

In [6]:
# stratified_sample.to_parquet("cic_ton_iot_5_percent.parquet")

In [7]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack
80330,192.168.10.16-40.68.232.24-59564-443-6,40.68.232.24,443.0,192.168.10.16,59564.0,6.0,03/07/2017 10:45:44,49.0,2.0,0.0,...,0.00000,0.00000,0.0,0.0,0.000,0.000,0.0,0.0,0,BENIGN
381309,192.168.10.9-38.65.20.26-1063-80-6,38.65.20.26,80.0,192.168.10.9,1063.0,6.0,03/07/2017 10:10:25,82.0,1.0,1.0,...,0.00000,0.00000,0.0,0.0,0.000,0.000,0.0,0.0,0,BENIGN
158503,192.168.10.14-23.208.163.76-54696-443-6,192.168.10.14,54696.0,23.208.163.76,443.0,6.0,4/7/2017 2:01,116004789.0,28.0,26.0,...,41302.83333,37147.18675,158089.0,23037.0,9625762.667,1298028.144,10000000.0,5503971.0,0,BENIGN
425716,192.168.10.25-52.52.10.28-57286-443-6,192.168.10.25,57286.0,52.52.10.28,443.0,6.0,03/07/2017 11:11:53,277.0,3.0,1.0,...,0.00000,0.00000,0.0,0.0,0.000,0.000,0.0,0.0,0,BENIGN
96495,192.168.10.3-192.168.10.8-53-60719-17,192.168.10.8,60719.0,192.168.10.3,53.0,17.0,6/7/2017 10:46,142.0,2.0,2.0,...,0.00000,0.00000,0.0,0.0,0.000,0.000,0.0,0.0,0,BENIGN


In [8]:
df.Label.value_counts()

Label
0    1136392
1     423280
Name: count, dtype: int64

In [9]:
drop_columns=["Flow ID", "Src IP", "Dst IP", "Timestamp", "Src Port", "Dst Port", "Attack"]
# drop_columns=["L4_SRC_PORT", "L4_DST_PORT", "Attack"]

src_ip_col="Src IP"
src_port_col="Src Port"
dst_ip_col="Dst IP"
dst_port_col="Dst Port"
timestamp_col="Timestamp"
timestamp_format="%d/%m/%Y %I:%M:%S %p"
label_col="Label"
# label_col="Label"
class_col="Attack"
class_num_col= "Class"

weak_columns = ['Flow Duration', 'Tot Bwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Mean', 'Bwd Pkt Len Max', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Pkts/s', 'Flow IAT Mean',
                'Flow IAT Max', 'Fwd IAT Mean', 'Bwd IAT Mean', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Size Avg', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Active Mean', 'Idle Mean']

# weak_columns = []

In [10]:
df[src_ip_col] = df[src_ip_col].apply(str)
df[src_port_col] = df[src_port_col].apply(str)
df[dst_ip_col] = df[dst_ip_col].apply(str)
df[dst_port_col] = df[dst_port_col].apply(str)

In [11]:
# df[src_ip_col] = df[src_ip_col].apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))
# df.head()

In [12]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(axis=0, how='any', inplace=True)
df.drop_duplicates(subset=None, keep="first", inplace=True)
df.reset_index(drop=True, inplace=True)

In [13]:
# X = df.drop([label_col], axis=1)
y = df[label_col]
X_train, X_test, y_train, y_test = train_test_split(
        df, y, test_size=0.3, random_state=13, stratify=y)

del df

In [14]:
X_train.shape

(989998, 84)

In [15]:
X_test.shape

(424286, 84)

In [16]:
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns ))  - set(list([label_col])) - set(drop_columns)  - set(weak_columns))

In [17]:
scaler = StandardScaler()
X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])

X_train['h'] = X_train[ cols_to_norm ].values.tolist()

In [18]:
G = nx.from_pandas_edgelist(X_train, src_ip_col, dst_ip_col, ['h',label_col], create_using=nx.MultiDiGraph())

In [19]:
del X_train

G = from_networkx(G,edge_attrs=['h',label_col] )

In [20]:
num_features = G.edata['h'].shape[1]

In [21]:
G.ndata['h'] = th.ones(G.num_nodes(), num_features)

G.ndata['h'] = th.reshape(G.ndata['h'], (G.ndata['h'].shape[0], 1, num_features))
G.edata['h'] = th.reshape(G.edata['h'], (G.edata['h'].shape[0], 1, num_features))

G.edata['train_mask'] = th.ones(len(G.edata['h']), dtype=th.bool)

In [22]:
class Model(nn.Module):
    def __init__(self, ndim_in, edim, ndim_out, activation, dropout):
        super().__init__()
        self.gnn = SAGE(ndim_in, edim, ndim_out, activation, dropout)
        self.pred = MLPPredictor(ndim_out, edim, 2)
    def forward(self, g, nfeats, efeats):
        h = self.gnn(g, nfeats, efeats)
        return self.pred(g, h)

In [23]:
class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
        super(SAGELayer, self).__init__()
        ### force to outut fix dimensions
        self.W_msg = nn.Linear(ndim_in + edims, ndim_out)
        ### apply weight
        self.W_apply = nn.Linear(ndim_in + ndim_out, ndim_out)
        self.activation = activation

    def message_func(self, edges):
        return {'m': self.W_msg(th.cat([edges.src['h'], edges.data['h']], 2))}

    def forward(self, g_dgl, nfeats, efeats):
        with g_dgl.local_scope():
            g = g_dgl
            g.ndata['h'] = nfeats
            g.edata['h'] = efeats
            # Eq4
            g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
            # Eq5          
            g.ndata['h'] = F.relu(self.W_apply(th.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))
            return g.ndata['h']


class SAGE(nn.Module):
    def __init__(self, ndim_in, edim, ndim_out, activation, dropout):
        super(SAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(SAGELayer(ndim_in, edim, 128, activation))
        self.layers.append(SAGELayer(128, edim, ndim_out, activation))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, g, nfeats, efeats):
        for i, layer in enumerate(self.layers):
            if i != 0:
                nfeats = self.dropout(nfeats)
            nfeats = layer(g, nfeats, efeats)
        return nfeats.sum(1)

In [24]:
class MLPPredictor(nn.Module):
    def __init__(self, in_features, edim, out_classes):
        super().__init__()
        if residual:
            self.W = nn.Linear(in_features * 2 + edim, out_classes)
        else:
            self.W = nn.Linear(in_features * 2, out_classes)
            

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
    
        if residual:
            h_uv = edges.data['h']
            h_uv = h_uv.view(h_uv.shape[0], h_uv.shape[2])
            score = self.W(th.cat([h_u, h_v, h_uv], 1))
        else:
            score = self.W(th.cat([h_u, h_v], 1))
            
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [25]:
# G.edata[label_col].cpu().numpy()

In [26]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(G.edata[label_col].cpu().numpy()),
                                                 y = G.edata[label_col].cpu().numpy())

In [27]:
# class_weights = th.FloatTensor(class_weights).cuda()
class_weights = th.FloatTensor(class_weights)

criterion = nn.CrossEntropyLoss(weight = class_weights)

In [28]:
def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()

In [29]:
node_features = G.ndata['h']
edge_features = G.edata['h']

edge_label = G.edata[label_col]
train_mask = G.edata['train_mask']

model = Model(num_features, num_features, 128, F.relu, 0.2)
opt = th.optim.Adam(model.parameters())


In [30]:
G.ndata['h']

tensor([[[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        ...,

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]]])

In [34]:
G.edata['h']

tensor([[[-0.0254, -0.3389, -0.2356,  ..., -0.2726, -0.0072,  0.0021]],

        [[-0.0330, -0.3389, -0.2356,  ..., -0.2726, -0.0099,  0.0021]],

        [[ 0.0088, -0.3389,  0.0066,  ..., -0.2726,  0.0021,  0.0021]],

        ...,

        [[-0.0364, -0.3389, -0.2356,  ..., -0.2726, -0.0099,  0.0021]],

        [[-0.0364, -0.3389, -0.2356,  ..., -0.2726, -0.0099,  0.0021]],

        [[-0.0364, -0.3389, -0.2356,  ..., -0.2726, -0.0099,  0.0021]]])

In [31]:
train_mask.shape

torch.Size([989998])

In [32]:
pred = model(G, node_features,edge_features)
loss = criterion(pred[train_mask] ,edge_label[train_mask])
opt.zero_grad()
loss.backward()
opt.step()

In [33]:
G.ndata['h']

tensor([[[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        ...,

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]]])

In [ ]:
for epoch in range(1,num_epochs):
#     pred = model(G, node_features,edge_features).cuda()
    pred = model(G, node_features,edge_features)
    loss = criterion(pred[train_mask] ,edge_label[train_mask])
    opt.zero_grad()
    loss.backward()
    opt.step()
    print('Epoch:', epoch ,' Training acc:', compute_accuracy(pred[train_mask], edge_label[train_mask]))

In [ ]:
del G

# Testing

In [ ]:
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])

In [ ]:
X_test['h'] = X_test[ cols_to_norm ].values.tolist()

In [ ]:
G_test = nx.from_pandas_edgelist(X_test, src_ip_col, dst_ip_col, ['h',label_col],create_using=nx.MultiDiGraph())
del X_test

In [ ]:
G_test = from_networkx(G_test,edge_attrs=['h',label_col] )

In [ ]:
actual = G_test.edata.pop(label_col)
G_test.ndata['feature'] = th.ones(G_test.num_nodes(),num_features)

In [ ]:
G_test.ndata['feature'] = th.reshape(G_test.ndata['feature'], (G_test.ndata['feature'].shape[0], 1, G_test.ndata['feature'].shape[1]))
G_test.edata['h'] = th.reshape(G_test.edata['h'], (G_test.edata['h'].shape[0], 1, G_test.edata['h'].shape[1]))

In [ ]:
# G_test = G_test.to('cuda:0')

In [ ]:
import timeit
start_time = timeit.default_timer()
node_features_test = G_test.ndata['feature']
edge_features_test = G_test.edata['h']
# test_pred = model(G_test, node_features_test, edge_features_test).cuda()
test_pred = model(G_test, node_features_test, edge_features_test)
elapsed = timeit.default_timer() - start_time

In [ ]:
print(str(elapsed) + ' seconds')

In [ ]:
test_pred = test_pred.argmax(1)
test_pred = th.Tensor.cpu(test_pred).detach().numpy()

In [ ]:
actual = ["Normal" if i == 0 else "Attack" for i in actual]
test_pred = ["Normal" if i == 0 else "Attack" for i in test_pred]

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(12, 12))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix(cm = confusion_matrix(actual, test_pred), 
                      normalize    = False,
                      target_names = np.unique(actual),
                      title        = "Confusion Matrix")

In [ ]:
print(classification_report(actual, test_pred, digits=4))

In [ ]:
# th.save(model.state_dict(), 'ton-iot.binary.pt')

In [ ]:
# from sklearn.metrics import 
# print(classification_report(actual, test_pred, digits=2))